# Supplementary Material

This is a companion piece to all the code lines in the paper

Listing: Installation of the package

In [ ]:
!pip install metacountregressor --upgrade
!pip install matplotlib --upgrade

Listing: Importing 

In [ ]:
from metacountregressor.solution import ObjectiveFunction
from metacountregressor.metaheuristics import (harmony_search,differential_evolution,simulated_annealing)         
from metacountregressor import helperprocess
import pandas as pd

Listing: Loading in data (example syntax)

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv("d_rp.csv")
y = df['Y']  # Frequency of crashes
X = df.drop(columns=['Y']) # setup X based on data
X.columns

In [ ]:
import os

from pathlib import Path

# Get the current notebook's directory
notebook_directory = Path().resolve()

# Set the working directory to the notebook's location
os.chdir(notebook_directory)

print(f"Current working directory: {os.getcwd()}")

Listing: Defining and Offset Variable

In [ ]:
#X['Offset'] = X['LENGTH']*np.log1p(X['AADT']) #Modify Here for Desired Offset
#X = X.drop(columns=['LENGTH', 'AADT'])  
y_name ='FREQ'
offset_name = 'Offset'
group_name = None
panel_name = None

# grabbing the offset amount
X['Offset'] = 1/5 #Modify Here for Desired Offset


X = X.dropna(how='any')


customising the arguments for objective and metaheuristic

In [ ]:
arguments = {'test_percentage': 0.3, 'complexity_level': 3, 'reg_penalty':0, 'MAX_TIME':6} #Objective args
arguments_hs = {'_par': 0.3, '_hms': 20}
arguments_sa = None #Note: Supply the relevant arguments, otherwise default arguments will be used
arguments_de = None

Defining the objective function for the available metaheuristics

In [ ]:
initial_solution = None
obj_fun = ObjectiveFunction(X, y, **arguments)
#perform harmony search
results_hs = harmony_search(obj_fun, initial_solution, **arguments_hs)
#perform differential evolution
''' Commenting out as only one metaheuristic should be used at a time, feel free to test the others
results_de = differential_evolution(obj_fun, initial_solution, **arguments_de)
#perform simulated annealing
results_sa = simulated_annealing(obj_fun, initial_solution, **arguments_sa)'
'''

Prespecifying an Intitial Solution

In [ ]:
manual_fit_spec = {
'fixed_terms': ['X1', 'X2'],
'rdm_terms':  ['X3:uniform', 'X4:uniform', 'X5:normal'],
'rdm_cor_terms': [],
'grouped_terms': [],
'hetro_in_means': [],
'transformations': ['no', 'no', 'no', 'no', 'no'],
'dispersion': 0
}

arguments['Manual_Fit'] = manual_fit_spec

obj_fun = ObjectiveFunction(X, y, **arguments)



Hetrogeienity in the means

In [ ]:
import pandas as pd
df = pd.read_csv("d_hm.csv")
y = df['Y']  # Frequency of crashes
X = df.drop(columns=['Y']) # setup X based on data
X.columns
print(X.columns)

manual_fit_spec = {
'fixed_terms': ['const'],
'rdm_terms':  [ 'X1:uniform'],
'rdm_cor_terms': [],
'grouped_terms': [],
'hetro_in_means': ['X2:normal', 'Z1:normal','X3:normal'],
'transformations': ['no', 'no', 'no', 'no', 'no', 'no'],
'dispersion': 0
}
arguments = {'test_percentage': 0.2, 'complexity_level': 3, 'reg_penalty':0} #Objective args
arguments['Manual_Fit'] = manual_fit_spec
initial_solution = None
obj_fun = ObjectiveFunction(X, y, **arguments)



Grouped random paramaters

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/zahern/data/refs/heads/main/rural_int.csv")
y = df['crashes']  # Frequency of crashes

df.drop(columns=[ 'year', 'orig_ID',
                                    'jurisdiction', 'town', 'maint_region', 'weather_station', 'dummy_winter_2', 'month', 'inj.fat', 'PDO', 'zonal_ID', 'ln_AADT', 'ln_seg'], inplace=True)  # was dropped postcode

           

arguments_hs = {'_par': 0.3, '_hms': 20}
arguments = {'test_percentage': 0.2, 'complexity_level': 5, 'reg_penalty':0} #Objective args
# Step 2: Process Data
model_terms = {
    'Y': 'crashes',         # Dependent variable
    'group': 'county',       # Grouping column (if any)
    'panels': 'element_ID',      # Panel column (if any)
    'Offset': None       # Offset column (if any)
}


X = df.drop(columns=['crashes']) # setup X based on data
X.columns
print(X.columns)

manual_fit_spec = {
'fixed_terms': ['const', 'DP10'],
'rdm_terms':  [ 'DX32:normal'],
'rdm_cor_terms': [],
'group_rdm': ['DPO1:triangular'],
'hetro_in_means': [],
'transformations': ['no', 'no', 'no', 'no', 'no', 'no'],
'dispersion': 0
}
arguments = {'test_percentage': 0.2, 'complexity_level': 6, 'reg_penalty':0, 'group':'county', 'panels':'element_ID'} #Objective args
arguments['Manual_Fit'] = manual_fit_spec
#initial_solution = None
obj_fun = ObjectiveFunction(X, y, **arguments)
initial_solution = None
results_hs = harmony_search(obj_fun, initial_solution, **arguments_hs)

Constrained Search Versus Freedom

In [ ]:
# CONSTRAINED SEARCH
model_terms = {
    'Y': 'Y',         # Replace 'FREQ' with the name of your dependent variable
    'group': None,    # Replace 'group_column' with the name of your grouping column (or None if not used)
    'panels': None,      # Replace 'panel_column' with the name of your panel column (or None if not used)
    'Offset': None                # Replace None with the name of your offset column if not using one
}
variable_decisions = {
'X1': {'levels': [0,1], 'transformations': ['no'], 'distributions': []},
'X2': {'levels': [1, 2,5], 'transformations': ['no'], 'distributions': ['n', 't']},
'X3':{'levels': [0, 2,5], 'transformations': ['no'], 'distributions': ['u', 'ln', 'tn']},
'Z1': {'levels': [0,5], 'transformations': ['no'], 'distributions': ['u', 'ln', 'tn']},
'Z2': {'levels': [0,1,5], 'transformations': ['no'], 'distributions': ['ln']}
}

a_des, X = helperprocess.set_up_analyst_constraints(X, model_terms, variable_decisions)

arguments['decisions'] = a_des
arguments['model_types'] = [[0]]
arguments['instance'] = 'constrained' # GIVE NAME
arguments['algorithm']='hs'
arguments_unconstrained = arguments.copy()
arguments_unconstrained['decisions'] = None
arguments_unconstrained['instance'] = 'unconstrained'
obj_fun = ObjectiveFunction(X, y, **arguments)
results = harmony_search(obj_fun)
helperprocess.results_printer(results, arguments['algorithm'], int(arguments['is_multi']))



In [ ]:
obj_fun = ObjectiveFunction(X, y, **arguments_unconstrained)
results = harmony_search(obj_fun)
helperprocess.results_printer(results, arguments_unconstrained['algorithm'], int(arguments_unconstrained['is_multi']))